In [1]:
import pandas as pd

# Load your dataset
file_path = "Dataset 1.csv"
df = pd.read_csv(file_path)

# Show shape, columns, and first 5 rows
print("Shape:", df.shape)
print("\nColumns:", df.columns.tolist())
print("\nSample rows:\n", df.head())
print("\nMissing values:\n", df.isnull().sum())
# Parse datetime column and set as index
df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')

# Check for parsing errors (any NaT values)
parsing_errors = df['datetime'].isnull().sum()
print(f"Datetime parsing errors: {parsing_errors}")

# Set as index for time-series analysis
df = df.set_index('datetime')

# Check for duplicate timestamps
duplicate_timestamps = df.index.duplicated().sum()
print(f"Duplicate timestamps: {duplicate_timestamps}")

# Show the first few rows with the new index
print(df.head())
# Solar Power Plant Data Column Grouping and Tagging

import pandas as pd

# --- Load the CSV and parse datetime ---
csv_path = 'Dataset 1.csv'  # <-- Update path if needed
df = pd.read_csv(csv_path)
df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')
df = df.set_index('datetime')  # Set datetime as index

# --- Column Group Definitions ---
column_groups = {
    # Meteorological Irradiance
    "irradiance_meteorological": [
        'meteorolgicas_em_03_02_gii', 'meteorolgicas_em_08_01_gii',
        'meteorolgicas_em_03_02_ghi', 'meteorolgicas_em_08_01_ghi',
        'meteorolgicas_em_08_01_gii_rear', 'meteorolgicas_em_03_02_gii_rear',
    ],
    "irradiance_cells": [
        'celulas_ctin08_cc_08_1_ir_cel_1', 'celulas_ctin08_cc_08_2_ir_cel_1',
        'celulas_ctin03_cc_03_1_ir_cel_1', 'celulas_ctin03_cc_03_2_ir_cel_1',
        'celulas_ctin08_cc_08_2_ir_cel_2', 'celulas_ctin03_cc_03_2_ir_cel_2',
        'celulas_ctin03_cc_03_1_ir_cel_2', 'celulas_ctin08_cc_08_1_ir_cel_2',
    ],
    "tracking_deviation": [
        'meteorolgicas_em_03_02_desviacin_incidente', 'meteorolgicas_em_08_01_desviacin_incidente'
    ],
    "temperature_ambient": [
        'meteorolgicas_em_03_02_t_amb', 'meteorolgicas_em_08_01_t_amb',
        'celulas_ctin08_cc_08_2_t_amb', 'celulas_ctin03_cc_03_1_t_amb',
        'celulas_ctin08_cc_08_1_t_amb', 'celulas_ctin03_cc_03_2_t_amb',
    ],
    "temperature_module": [
        'celulas_ctin03_cc_03_1_t_mod', 'celulas_ctin08_cc_08_2_t_mod',
        'celulas_ctin03_cc_03_2_t_mod', 'celulas_ctin08_cc_08_1_t_mod',
    ],
    "temperature_datalogger": [
        'meteorolgicas_em_03_02_t_dlogger', 'meteorolgicas_em_08_01_t_dlogger'
    ],
    "humidity": [
        'meteorolgicas_em_03_02_h_r', 'meteorolgicas_em_08_01_h_r'
    ],
    "wind_speed": [
        'meteorolgicas_em_08_01_ws', 'meteorolgicas_em_03_02_ws',
        'celulas_ctin03_cc_03_2_wind_speed', 'celulas_ctin08_cc_08_1_wind_speed',
        'celulas_ctin08_cc_08_2_wind_speed', 'celulas_ctin03_cc_03_1_wind_speed',
    ],
    "wind_direction": [
        'meteorolgicas_em_08_01_wd', 'meteorolgicas_em_03_02_wd'
    ],
    "string_voltage": [col for col in df.columns if 'pv_v' in col],
    "string_current_ct03_s8": [
        f'inversores_ctin03_strings_string8_pv_i{i}' for i in range(1, 14)
    ],
    "string_current_ct03_s10": [
        f'inversores_ctin03_strings_string10_pv_i{i}' for i in range(1, 14)
    ],
    "string_current_ct08_s9": [
        f'inversores_ctin08_strings_string9_pv_i{i}' for i in range(1, 14)
    ],
    "string_current_ct08_s12": [
        f'inversores_ctin08_strings_string12_pv_i{i}' for i in range(1, 11)
    ],
    "inverter_power_ac": [
        'inversores_ctin08_inv_08_08_p', 'inversores_ctin03_inv_03_03_p'
    ],
    "inverter_power_dc": [
        'inversores_ctin03_inv_03_03_p_dc', 'inversores_ctin08_inv_08_08_p_dc'
    ],
    "inverter_energy": [
        'inversores_ctin03_inv_03_03_eact_tot', 'inversores_ctin08_inv_08_08_eact_tot'
    ],
    "tracker_position": [
        'seguidores_ct08_gcu081_t0808029_pos_obj', 'seguidores_ct08_gcu081_t0808029_pos_ang',
        'seguidores_ct03_gcu031_t0308035_pos_ang', 'seguidores_ct03_gcu031_t0308035_pos_obj'
    ],
    "tracker_mode": [
        'seguidores_ct08_gcu081_t0808029_workingmode', 'seguidores_ct03_gcu031_t0308035_workingmode'
    ],
    "plant_power_control": [
        'ppc_consig_p', 'ppc_p_tot', 'ppc_eact_export', 'ppc_eact_imp'
    ],
    "theoretical_power": [
        'ttr_potenciaproducible'
    ]
}

# --- Analysis Group Definitions ---
analysis_groups = {
    "all_irradiance": column_groups["irradiance_meteorological"] + column_groups["irradiance_cells"],
    "all_temperature": column_groups["temperature_ambient"] + column_groups["temperature_module"] + column_groups["temperature_datalogger"],
    "all_wind": column_groups["wind_speed"] + column_groups["wind_direction"],
    "all_string_electrical": (
        column_groups["string_current_ct03_s8"] +
        column_groups["string_current_ct03_s10"] +
        column_groups["string_current_ct08_s9"] +
        column_groups["string_current_ct08_s12"] +
        column_groups["string_voltage"]
    ),
    "all_power": (
        column_groups["inverter_power_ac"] +
        column_groups["inverter_power_dc"] +
        column_groups["plant_power_control"] +
        column_groups["theoretical_power"]
    ),
    "all_tracker": column_groups["tracker_position"] + column_groups["tracker_mode"],
    "environmental": column_groups["temperature_ambient"] + column_groups["humidity"] + column_groups["wind_speed"] + column_groups["wind_direction"],
    "soiling_analysis": column_groups["irradiance_cells"] + column_groups["irradiance_meteorological"],
    "tracking_analysis": column_groups["tracking_deviation"] + column_groups["tracker_position"] + column_groups["tracker_mode"]
}

# --- Functions for Column Access ---
def get_columns(group_name):
    """Return existing columns for the given group name."""
    if group_name in column_groups:
        return [col for col in column_groups[group_name] if col in df.columns]
    elif group_name in analysis_groups:
        return [col for col in analysis_groups[group_name] if col in df.columns]
    else:
        print(f"Group '{group_name}' not found.")
        return []

def get_subset_df(group_name, include_datetime=True):
    """Return a subset DataFrame for a given group."""
    cols = get_columns(group_name)
    if include_datetime:
        # Include datetime as index or column
        return df[cols].copy().reset_index()  # Datetime becomes a column
    else:
        return df[cols].copy()

# --- Display Column Group Info ---
print("Available column groups:")
for group in column_groups:
    real_cols = get_columns(group)
    print(f"  - {group}: {len(real_cols)} columns")

print("\nAvailable analysis groups:")
for group in analysis_groups:
    real_cols = get_columns(group)
    print(f"  - {group}: {len(real_cols)} columns")

# --- Example Usages ---
irradiance_df = get_subset_df("all_irradiance")
print(f"\nIrradiance dataframe shape: {irradiance_df.shape}")

power_df = get_subset_df("all_power")
print(f"Power dataframe shape: {power_df.shape}")

env_df = get_subset_df("environmental")
print(f"Environmental dataframe shape: {env_df.shape}")
# 1. Show missing data counts for all columns
missing_counts = df.isnull().sum().sort_values(ascending=False)
print("Missing values per column:\n", missing_counts[missing_counts > 0])

# 2. Show missing rates per group
print("\nMissing value summary by group:")
for group in column_groups:
    cols = get_columns(group)
    if cols:
        missing_rate = df[cols].isnull().mean().mean() * 100
        print(f"  - {group}: {missing_rate:.2f}% average missing")

# --- 1. Drop columns with very high missingness (>95%) ---
high_missing_cols = [col for col in df.columns if df[col].isnull().mean() > 0.95]
df_cleaned = df.drop(columns=high_missing_cols)
print(f"Dropped columns with >95% missing: {high_missing_cols}")

# --- 2. Interpolate and fill low-missing columns (≤5%) ---
low_missing_cols = [col for col in df_cleaned.columns if df_cleaned[col].isnull().mean() <= 0.05]
df_cleaned[low_missing_cols] = (
    df_cleaned[low_missing_cols]
    .interpolate(method='linear', limit_direction='both')
    .fillna(method='ffill')
    .fillna(method='bfill')
)

# --- 3. Identify columns with moderate missingness (50–95%) ---
mid_missing_cols = [col for col in df_cleaned.columns if 0.5 < df_cleaned[col].isnull().mean() <= 0.95]
print(f"Columns with 50–95% missing (inspect before deciding): {mid_missing_cols}")

# --- 4. Remove night data for operational analysis ---
irr_col = 'meteorolgicas_em_03_02_ghi'  # Main irradiance column
day_df = df_cleaned[df_cleaned[irr_col] > 5].copy()  # Keep rows with GHI > 5 W/m²
print(f"Shape after removing night data: {day_df.shape}")

# Now, day_df is ready for most solar performance and loss analyses!
# Total rows before filtering
total_rows = df_cleaned.shape[0]

# Total rows after filtering (daytime only)
day_rows = day_df.shape[0]

# Nighttime rows dropped
night_rows = total_rows - day_rows
night_pct = night_rows / total_rows * 100

print(f"Total rows before filtering: {total_rows}")
print(f"Total rows after removing night data: {day_rows}")
print(f"Nighttime rows dropped: {night_rows} ({night_pct:.2f}%)")
# --- Basic Outlier Removal for Solar PV Data ---

def clean_outliers(df, col, lower, upper):
    """
    Sets values outside [lower, upper] to NaN for a given column.
    Prints how many values were set to NaN.
    """
    if col not in df.columns:
        return df
    before = df[col].isnull().sum()
    df[col] = df[col].where(df[col].between(lower, upper), float('nan'))
    after = df[col].isnull().sum()
    print(f"{col}: set {after-before} outliers to NaN")
    return df

# --- Set physical ranges per group (customize for your plant if needed) ---
outlier_limits = {
    'irradiance': (0, 1400),       # W/m²
    'temperature': (-10, 80),      # °C
    'wind_speed': (0, 40),         # m/s
    'humidity': (0, 100),          # %
    'power': (0, 48000),           # kW (48 MW plant, change if needed)
    'string_voltage': (0, 1500),   # V
    'string_current': (0, 40),     # A
}

# --- Apply limits to relevant columns using your grouping functions ---

# Irradiance
for col in get_columns('all_irradiance'):
    day_df = clean_outliers(day_df, col, *outlier_limits['irradiance'])

# Temperature
for col in get_columns('all_temperature'):
    day_df = clean_outliers(day_df, col, *outlier_limits['temperature'])

# Wind Speed
for col in get_columns('wind_speed'):
    day_df = clean_outliers(day_df, col, *outlier_limits['wind_speed'])

# Humidity
for col in get_columns('humidity'):
    day_df = clean_outliers(day_df, col, *outlier_limits['humidity'])

# Power (AC/DC/Theoretical)
for col in get_columns('all_power'):
    day_df = clean_outliers(day_df, col, *outlier_limits['power'])

# String Voltages
for col in get_columns('string_voltage'):
    day_df = clean_outliers(day_df, col, *outlier_limits['string_voltage'])

# String Currents (all current groups)
string_current_cols = (
    get_columns('string_current_ct03_s8') +
    get_columns('string_current_ct03_s10') +
    get_columns('string_current_ct08_s9') +
    get_columns('string_current_ct08_s12')
)
for col in string_current_cols:
    day_df = clean_outliers(day_df, col, *outlier_limits['string_current'])

# --- Optional: Interpolate/fill after outlier removal if needed ---
# day_df = day_df.interpolate(method='linear', limit_direction='both').fillna(method='ffill').fillna(method='bfill')

print("Basic outlier cleaning complete!")
# Fill all remaining NaN values in the cleaned daytime dataframe
day_df = (
    day_df
    .interpolate(method='linear', limit_direction='both')
    .fillna(method='ffill')
    .fillna(method='bfill')
)

# Check if any NaNs remain
nan_count = day_df.isnull().sum().sum()
print(f"Total NaNs remaining after filling: {nan_count}")

# =============================================================================
# SAVE PREPROCESSED DATA TO CSV FILE
# =============================================================================
print("\n=== SAVING PREPROCESSED DATA ===")

# Save the cleaned and preprocessed daytime data to CSV
output_filename = "preprocessed_solar_data.csv"
day_df.to_csv(output_filename)
print(f"✅ Preprocessed data saved to: {output_filename}")
print(f"   Shape: {day_df.shape}")
print(f"   Date range: {day_df.index.min()} to {day_df.index.max()}")

# Optional: Also save a summary of the preprocessing steps
summary_filename = "preprocessing_summary.txt"
with open(summary_filename, 'w') as f:
    f.write("SOLAR POWER PLANT DATA PREPROCESSING SUMMARY\n")
    f.write("=" * 50 + "\n\n")
    f.write(f"Original dataset shape: {df.shape}\n")
    f.write(f"High missing columns dropped (>95%): {len(high_missing_cols)}\n")
    f.write(f"Night data rows removed: {night_rows} ({night_pct:.2f}%)\n")
    f.write(f"Final preprocessed shape: {day_df.shape}\n")
    f.write(f"Remaining NaN values: {nan_count}\n")
    f.write(f"Date range: {day_df.index.min()} to {day_df.index.max()}\n")
    f.write(f"\nPreprocessed data saved to: {output_filename}\n")

print(f"✅ Preprocessing summary saved to: {summary_filename}")

# =============================================================================

Shape: (14172, 109)

Columns: ['datetime', 'meteorolgicas_em_03_02_gii', 'meteorolgicas_em_08_01_gii', 'meteorolgicas_em_03_02_ghi', 'meteorolgicas_em_08_01_ghi', 'meteorolgicas_em_08_01_gii_rear', 'meteorolgicas_em_03_02_gii_rear', 'meteorolgicas_em_03_02_desviacin_incidente', 'meteorolgicas_em_08_01_desviacin_incidente', 'meteorolgicas_em_03_02_t_amb', 'meteorolgicas_em_08_01_t_amb', 'meteorolgicas_em_03_02_h_r', 'meteorolgicas_em_08_01_h_r', 'meteorolgicas_em_03_02_t_dlogger', 'meteorolgicas_em_08_01_t_dlogger', 'meteorolgicas_em_08_01_ws', 'meteorolgicas_em_03_02_ws', 'meteorolgicas_em_08_01_wd', 'meteorolgicas_em_03_02_wd', 'celulas_ctin08_cc_08_1_ir_cel_1', 'celulas_ctin08_cc_08_2_ir_cel_1', 'celulas_ctin03_cc_03_1_ir_cel_1', 'celulas_ctin03_cc_03_2_ir_cel_1', 'celulas_ctin08_cc_08_2_ir_cel_2', 'celulas_ctin03_cc_03_2_ir_cel_2', 'celulas_ctin03_cc_03_1_ir_cel_2', 'celulas_ctin08_cc_08_1_ir_cel_2', 'celulas_ctin08_cc_08_2_t_amb', 'celulas_ctin03_cc_03_1_t_amb', 'celulas_ctin08_cc_

/tmp/ipython-input-1-4245071371.py:201: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .fillna(method='ffill')
/tmp/ipython-input-1-4245071371.py:202: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .fillna(method='bfill')


Columns with 50–95% missing (inspect before deciding): ['inversores_ctin03_strings_string8_pv_v', 'inversores_ctin03_strings_string8_pv_i9', 'inversores_ctin03_strings_string8_pv_i13', 'inversores_ctin03_strings_string8_pv_i1', 'inversores_ctin03_strings_string8_pv_i6', 'inversores_ctin03_strings_string8_pv_i4', 'inversores_ctin03_strings_string8_pv_i11', 'inversores_ctin03_strings_string8_pv_i10', 'inversores_ctin03_strings_string8_pv_i2', 'inversores_ctin03_strings_string8_pv_i5', 'inversores_ctin03_strings_string8_pv_i7', 'inversores_ctin03_strings_string8_pv_i12', 'inversores_ctin03_strings_string8_pv_i3', 'inversores_ctin03_strings_string8_pv_i8', 'inversores_ctin03_strings_string10_pv_i9', 'inversores_ctin03_strings_string10_pv_i7', 'inversores_ctin03_strings_string10_pv_i4', 'inversores_ctin03_strings_string10_pv_i12', 'inversores_ctin03_strings_string10_pv_i8', 'inversores_ctin03_strings_string10_pv_i13', 'inversores_ctin03_strings_string10_pv_i10', 'inversores_ctin03_strings_s

/tmp/ipython-input-1-4245071371.py:298: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .fillna(method='ffill')
/tmp/ipython-input-1-4245071371.py:299: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  .fillna(method='bfill')


✅ Preprocessed data saved to: preprocessed_solar_data.csv
   Shape: (6804, 106)
   Date range: 2024-10-01 08:00:00 to 2025-03-16 12:45:00
✅ Preprocessing summary saved to: preprocessing_summary.txt


In [ ]:
import pandas as pd

# Load the data
file_path = "preprocessed_solar_data.csv"  # Adjust path if needed
df = pd.read_csv(file_path)

# Basic shape
print(f"Number of Rows: {df.shape[0]}")
print(f"Number of Columns: {df.shape[1]}\n")

# Data types
print("Column Data Types:")
print(df.dtypes.value_counts(), "\n")

# Null values
print("Null Values Per Column:")
print(df.isnull().sum(), "\n")

# Unique values
print("Unique Values Per Column:")
print(df.nunique(), "\n")

# Descriptive statistics for numerical columns
print("Descriptive Statistics:")
print(df.describe().T)

# Descriptive stats for object (categorical) columns
categorical_cols = df.select_dtypes(include='object').columns
if len(categorical_cols):
    print("\nCategorical Column Statistics:")
    print(df[categorical_cols].describe().T)
